<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Beto
[GitHub LINK](https://github.com/dccuchile/beto)

In [ ]:
import numpy as np
import tensorflow as tf

## 1 - BETO embeddings
Se necesita instalar la librería de "transformers" de Hugging Face para utilizar los modelos de BERT y sus funciones de ayuda

In [ ]:
!pip install transformers --quiet

In [ ]:
from transformers import TFBertModel, BertTokenizer

# Muy importante que para tensorflow los modelos Bert deben empezar con "TF"
# de lo contrario estaremos utilizando un modelo para pytorch

# Descargamos el modelo base de BETO y su correspondiente tokenizer (BERT para español)
model = TFBertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

In [ ]:
model.summary()

In [ ]:
max_length = 12
text = "hola mundo! soy beto"

input_dict = tokenizer(text,
                       add_special_tokens=True,
                       return_token_type_ids=True, # indican segmentación de dos textos como por ejemplo para entailment
                       return_attention_mask=True,
                       max_length=max_length, 
                       padding="max_length", 
                       truncation=True, 
                       return_tensors='tf')

# Idem con encode plus (mismo resultado)
# input_dict = tokenizer.encode_plus(
#     text,
#     add_special_tokens=True,
#     return_token_type_ids=False,
#     return_attention_mask=True,
#     max_length=max_length, # truncates if len(s) > max_length
#     padding="max_length",
#     truncation=True,
#     return_tensors='tf'
# )

print(input_dict.keys())
print("Inputs ids:", input_dict['input_ids'])
print("Attention mask:", input_dict['attention_mask'])
print("Token type ids:", input_dict['token_type_ids'])

# EL primer token es el de CLS
# EL ante último token es de SEP
# EL último token es de PAD

Se puede observar que el sistema está creando más tokens que palabras, esto es porque BERT agerga tokens especiales semánticos (separadores, conjugación, símbolos, etc).

In [ ]:
# Tokens transformados a Ids
print(input_dict['input_ids'])

In [ ]:
# Ids transformamos a tokens
for id in input_dict['input_ids'][0]:
    token = tokenizer.convert_ids_to_tokens(int(id))
    print(token)

__IMPORTANTE:__ "beto" no pudo ser tokenizada (no estaba en el vocabulario) por lo que el tokenizar la dividió en 2 palabras tokenizables.

In [ ]:
# Convertir todos los tokens descartando los especiales
tokens = tokenizer.convert_ids_to_tokens(input_dict['input_ids'][0], skip_special_tokens=True) 
tokens

In [ ]:
# Inferir con el modelo de Bert, nótese que el input es el conjunto de Ids y attention mask
X_ensayo = [input_dict['input_ids'], input_dict['attention_mask']]
out = model.predict(X_ensayo)
last_hidden_state, pooler_output = out[0], out[1]

In [ ]:
# Embedding de salida que representa toda la sentencia de entrada:
pooler_output.shape

In [ ]:
# Embedding de cada palabra/token de entrada
# (batch_size, sequence_length, hidden_size)
print("Embeddings shape:", last_hidden_state.shape)

## 2 - Interpretar cómo la tokenización transforma las palabras

In [ ]:
text2 = ["Acercame el banco para sentarme", 
         "Esto lo encontré debajo del banco de una plaza", 
         "Hoy tengo que ir al banco a hacer unos trámites",
         "Esa plata me la depositan directamente en el banco",
         "Yo no me banco que me hagan eso",
         "A ella la banco en lo que sea"]

input_dict2 = tokenizer(text2,
                    add_special_tokens=True,
                    return_token_type_ids=False,
                    return_attention_mask=True,
                    max_length=max_length, 
                    padding="max_length", 
                    truncation=True, 
                    return_tensors='tf')

print(input_dict2.keys())
print("Inputs ids:", input_dict2['input_ids'])
print("Attention mask:", input_dict2['attention_mask'])

In [ ]:
# Convertir todos los tokens descartando los especiales
for in_dict in input_dict2['input_ids']:
    print(tokenizer.convert_ids_to_tokens(in_dict, skip_special_tokens=True))

In [ ]:
X_ensayo2 = [input_dict2['input_ids'], input_dict2['attention_mask']]
out2 = model.predict(X_ensayo2)
last_hidden_state2, pooler_output2 = out2[0], out2[1]
last_hidden_state2.shape

In [ ]:
# El embedding de está es:
last_hidden_state2[0, 1, :]

## 3 - Comparar los embeddings

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import pairwise

def compare_embeddings(set_idx):
    embs = []
    words = []
    for i in range(len(last_hidden_state2)):
        token_id = int(input_dict2['input_ids'][i][set_idx[i]])
        token = tokenizer.convert_ids_to_tokens(token_id)
        words.append(token)
        embs.append(last_hidden_state2[i, set_idx[i], :])

    similarity_cosine = pairwise.cosine_similarity(embs)

    fig = plt.figure(figsize=(16,9))
    ax = fig.add_subplot()
    sns.heatmap(similarity_cosine, xticklabels=words, yticklabels=words,
     annot=True, fmt=".2f", cmap="YlGnBu", ax=ax, mask=np.triu(similarity_cosine))
    plt.show()

In [ ]:
# el índice para cada texto de `text2` en donde se encuentra la palabra "banco"
set_idx = [4,6,6,10,4,4]

In [ ]:
# Obtener la matriz de distancia coseno entre los embeddings 
compare_embeddings(set_idx)

## Conclusiones 
- Como estamos utilizando BERT "uncased" el sistema pasa a minúsculas al texto automaticamente. Debemos usar BETO cased para soportar mayúsculas.
- BERT soporta utilizar palabras con tílde y posee un token_id diferente para cada una.
- BERT no es muy bueno para obtener embeddings individuales de palabras (ELMo es mejor en ese aspecto o Glove/Fasttext). Lo utilizaremos para obtener el embedding de la sentencia o contexto para clasificación o comparación de textos.